Importamos las librerias a utilizar

In [1]:
import json
import pandas as pd
import ast
from pandas import json_normalize
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

Abrimos el archivo 'australian_user_reviews.json'

In [2]:
# Abrimos el json
rows=[]
with open("datasets/australian_user_reviews.json",encoding="utf-8")as file: 
    for line in file.readlines():
        rows.append(ast.literal_eval(line))

# Creamos el dataframe
reviews = pd.DataFrame(rows) 

In [3]:
# Visualizamos el estado de nuestro dataframe
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [4]:
# Desanidamos la columna reviews y mostramos el tamaño
reviews = pd.concat([pd.concat([pd.DataFrame(row['reviews']), pd.Series([row['user_id']] * len(row['reviews']), name='user_id')], axis=1) for _, row in reviews.iterrows()], ignore_index=True)
reviews.shape

(59305, 8)

In [5]:
# Sacamos los valores duplicados
reviews.drop_duplicates(inplace=True)
reviews.shape

(58431, 8)

In [6]:
# Revisamos la columna 'posted'
reviews['posted'].tail()

59300    Posted July 10.
59301     Posted July 8.
59302     Posted July 3.
59303    Posted July 20.
59304     Posted July 2.
Name: posted, dtype: object

Decidimos sacar las columnas funny, last_edited y Helpful para reducir tamaño, ya que no nos van a servir

In [7]:
# Sacamos las columnas
reviews = reviews.drop(columns=['funny', 'last_edited', 'helpful'])

In [8]:
# Revisamos el estado de 'posted'
reviews["posted"]

0         Posted November 5, 2011.
1            Posted July 15, 2011.
2           Posted April 21, 2011.
3            Posted June 24, 2014.
4        Posted September 8, 2013.
                   ...            
59300              Posted July 10.
59301               Posted July 8.
59302               Posted July 3.
59303              Posted July 20.
59304               Posted July 2.
Name: posted, Length: 58431, dtype: object

In [9]:
# Eliminar la palabra "Posted" de la columna 'posted'
reviews['posted'] = reviews['posted'].str.replace('Posted ', '')

# Convertir la columna 'posted' a formato de fecha
reviews['posted'] = pd.to_datetime(reviews['posted'], errors='coerce')

# Extraer solo el año de la fecha y lo usamos en una nueva columna llamada "posted_year"
reviews['posted_year'] = reviews['posted'].dt.year

# Borramos la columna "posted"
reviews = reviews.drop('posted', axis=1)

In [10]:
# Sacamos los valores nulos de la columna posted_year
reviews = reviews.dropna(subset=['posted_year'])
reviews['posted_year'] = reviews['posted_year'].astype(int)

In [11]:
# Revisamos el estado de nuestro dataframe
reviews.head()

,item_id,recommend,review,user_id,posted_year
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011
1,22200,True,It's unique and worth a playthrough.,76561197970982479,2011
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011
3,251610,True,I know what you think when you see this title ...,js41637,2014
4,227300,True,For a simple (it's actually not all that simpl...,js41637,2013


In [12]:
# Rellenamos los valores nulos de la columna 'review' con una cadena vacía
reviews['review'].fillna('', inplace=True)

C:\Users\juan_\AppData\Local\Temp\ipykernel_2920\1950861582.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reviews['review'].fillna('', inplace=True)


Creamos la funcion de Analisis de sentimiento

In [13]:
def analisis_sentimiento(sentimiento):
    """
    Realiza un análisis de sentimiento basado en el valor proporcionado.

    Parámetros:
    - sentimiento (float): El valor del sentimiento a analizar.

    Retorna:
    - int: 2 si el sentimiento es positivo, 0 si es negativo y 1 si es neutral.
    """
    # Sentimiento positivo
    if sentimiento >= 0.05:
        return 2 
    # Sentimiento negativo
    elif sentimiento <= -0.04:
        return 0
    # Sentimiento neutral
    else:
        return 1

In [14]:
# Inicializar el analizador de sentimientos de NLTK
sid = SentimentIntensityAnalyzer()

In [15]:
# Calcula el puntaje de sentimiento para cada review
reviews['sentiment_score'] = reviews['review'].apply(lambda x: sid.polarity_scores(x)['compound'])

# Realiza el análisis de sentimiento basado en el puntaje de sentimiento
reviews['sentiment_analysis'] = reviews['sentiment_score'].apply(analisis_sentimiento)

In [16]:
# Borramos las columnas review y sentiment_score, ya que no las vamos a necesitar mas
reviews.drop(columns=['review', 'sentiment_score'],inplace=True)

In [26]:
# Cambiamos el tipo de dato de la columna item_id a entero
reviews['item_id'] = reviews['item_id'].astype(int)

In [27]:
# Exportamos nuestro dataset
reviews.to_parquet("datasets/Parquet/reviews.parquet", index=False)